In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time
import math
import speech_recognition as sr
import pyaudio

dragging = False  

In [ ]:
import cv2
import mediapipe as mp
import pyautogui

cap = cv2.VideoCapture(0)

hand_detector = mp.solutions.hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
drawing_utils = mp.solutions.drawing_utils

screen_width, screen_height = pyautogui.size()


In [ ]:


prev_x, prev_y = 0, 0
smoothening = 5
dragging = False


In [ ]:
import cv2
import mediapipe as mp


cap = cv2.VideoCapture(0)  


mp_hands = mp.solutions.hands
hand_detector = mp_hands.Hands(
    max_num_hands=1, 
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

drawing_utils = mp.solutions.drawing_utils

while True:
    success, frame = cap.read()
    if not success:
        print("Failed to capture image")
        break

    frame = cv2.flip(frame, 1)
    
    frame_height, frame_width, _ = frame.shape

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    output = hand_detector.process(rgb_frame)
    hands = output.multi_hand_landmarks  

    if hands:
        for hand_landmarks in hands:
            drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Hand Landmarks Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

hand_detector = mp.solutions.hands.Hands(
    max_num_hands=1, 
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.7
)
drawing_utils = mp.solutions.drawing_utils

while True:
    success, frame = cap.read()
    if not success:
        print("Failed to capture image")
        break

    frame = cv2.flip(frame, 1)
    
    frame_height, frame_width, _ = frame.shape

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    output = hand_detector.process(rgb_frame)
    hands = output.multi_hand_landmarks

    if hands:
        for hand_landmarks in hands:
            drawing_utils.draw_landmarks(frame, hand_landmarks)
            landmarks = hand_landmarks.landmark

            index_finger = landmarks[8]
            thumb_finger = landmarks[4]
            
            index_x = int(index_finger.x * frame_width)
            index_y = int(index_finger.y * frame_height)
            thumb_x = int(thumb_finger.x * frame_width)
            thumb_y = int(thumb_finger.y * frame_height)

            cv2.circle(frame, (index_x, index_y), 10, (0, 255, 255), cv2.FILLED)
            cv2.circle(frame, (thumb_x, thumb_y), 10, (0, 255, 255), cv2.FILLED)

    cv2.imshow("Hand Landmarks and Finger Circles", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:

import cv2
import mediapipe as mp
import pyautogui

cap = cv2.VideoCapture(0)  

hand_detector = mp.solutions.hands.Hands(
    max_num_hands=1, 
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.7
)
drawing_utils = mp.solutions.drawing_utils

screen_width, screen_height = pyautogui.size()  

prev_x, prev_y = 0, 0

while True:
    success, frame = cap.read()
    if not success:
        print("Failed to capture image")
        break

    frame = cv2.flip(frame, 1)
    
    frame_height, frame_width, _ = frame.shape

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    output = hand_detector.process(rgb_frame)
    hands = output.multi_hand_landmarks  

    if hands:
        for hand_landmarks in hands:
            drawing_utils.draw_landmarks(frame, hand_landmarks)
            landmarks = hand_landmarks.landmark

            index_finger = landmarks[8]
            thumb_finger = landmarks[4]
            
            index_x = int(index_finger.x * frame_width)
            index_y = int(index_finger.y * frame_height)

            curr_x = prev_x + (index_x - prev_x) / smoothening
            curr_y = prev_y + (index_y - prev_y) / smoothening

            pyautogui.moveTo(curr_x * screen_width / frame_width, curr_y * screen_height / frame_height)

            prev_x, prev_y = curr_x, curr_y

    cv2.imshow("Hand Landmarks and Mouse Control", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time
import math
import numpy as np

cap = cv2.VideoCapture(0)
hands = mp.solutions.hands.Hands(max_num_hands=1, min_detection_confidence=0.8, min_tracking_confidence=0.8)
drawing = mp.solutions.drawing_utils
screen_width, screen_height = pyautogui.size()

prev_x, prev_y = 0, 0
alpha = 0.1
dead_zone = 15
dragging = False
cooldown = 2
last_back = last_vol_up = last_vol_down = 0

def distance(p1, p2, w, h):
    return math.hypot(int(p1.x * w) - int(p2.x * w), int(p1.y * h) - int(p2.y * h))

x_margin = 0.2
y_margin = 0.2

while True:
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)
    landmarks = result.multi_hand_landmarks

    if landmarks:
        for hand in landmarks:
            drawing.draw_landmarks(frame, hand)
            lm = hand.landmark

            index_tip = lm[8]
            middle_tip = lm[12]
            ring_tip = lm[16]
            thumb_tip = lm[4]

            index_x = int(index_tip.x * w)
            index_y = int(index_tip.y * h)
            cv2.circle(frame, (index_x, index_y), 10, (0, 255, 255), cv2.FILLED)

            if not (x_margin < index_tip.x < 1 - x_margin and y_margin < index_tip.y < 1 - y_margin):
                continue

            mx = np.clip((index_tip.x - x_margin) / (1 - 2 * x_margin), 0, 1)
            my = np.clip((index_tip.y - y_margin) / (1 - 2 * y_margin), 0, 1)
            target_x = int(mx * screen_width)
            target_y = int(my * screen_height)

            curr_x = (1 - alpha) * prev_x + alpha * target_x
            curr_y = (1 - alpha) * prev_y + alpha * target_y

            if abs(curr_x - prev_x) > dead_zone or abs(curr_y - prev_y) > dead_zone:
                pyautogui.moveTo(curr_x, curr_y)
                prev_x, prev_y = curr_x, curr_y

            now = time.time()
            thumb_index_dist = distance(thumb_tip, index_tip, w, h)
            thumb_ring_dist = distance(thumb_tip, ring_tip, w, h)

            if thumb_index_dist < 45:
                if not dragging:
                    pyautogui.mouseDown()
                    dragging = True
            else:
                if dragging:
                    pyautogui.mouseUp()
                    dragging = False

            if thumb_ring_dist < 45 and now - last_back > cooldown:
                pyautogui.hotkey('alt', 'left')
                last_back = now

            if index_tip.y < lm[6].y and middle_tip.y < lm[10].y and ring_tip.y < lm[14].y:
                if now - last_vol_up > cooldown:
                    pyautogui.press("volumeup")
                    last_vol_up = now

            if index_tip.y < lm[6].y and middle_tip.y > lm[10].y and ring_tip.y > lm[14].y:
                if now - last_vol_down > cooldown:
                    pyautogui.press("volumedown")
                    last_vol_down = now

    cv2.imshow("🖐 Gesture Mouse", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import pyautogui
import speech_recognition as sr
import time
import sys

recognizer = sr.Recognizer()
mic = sr.Microphone()

# Calibrate mic
with mic as source:
    print("🎤 Calibrating microphone... Please wait.")
    recognizer.adjust_for_ambient_noise(source)
    print("🎤 Microphone calibrated. You can speak now.")

# Voice command loop
while True:
    with mic as source:
        print("🎧 Listening for command...")
        try:
            audio = recognizer.listen(source, timeout=5)
            command = recognizer.recognize_google(audio).lower()
            print("🗣️ Command received:", command)

            # Mouse click commands
            if "click" in command:
                pyautogui.click()
            elif "double click" in command:
                pyautogui.doubleClick()
            elif "right click" in command:
                pyautogui.click(button='right')

            # Scroll commands
            elif "scroll up" in command:
                pyautogui.scroll(300)
            elif "scroll down" in command:
                pyautogui.scroll(-300)
            elif "scroll right" in command:
                pyautogui.keyDown('shift')
                pyautogui.scroll(-300)
                pyautogui.keyUp('shift')
            elif "scroll left" in command:
                pyautogui.keyDown('shift')
                pyautogui.scroll(300)
                pyautogui.keyUp('shift')

            # Volume commands
            elif "volume up" in command:
                pyautogui.press("volumeup")
            elif "volume down" in command:
                pyautogui.press("volumedown")

            # Zoom commands
            elif "zoom in" in command:
                pyautogui.hotkey('ctrl', '+')
            elif "zoom out" in command:
                pyautogui.hotkey('ctrl', '-')

            # Exit command
            elif "exit" in command or "close" in command:
                print("🛑 Voice command 'exit' detected. Exiting program...")
                pyautogui.alert("Voice Mouse Control: Exiting Program")
                sys.exit()

        except sr.WaitTimeoutError:
            print("⏱ Timeout: No voice input.")
        except sr.UnknownValueError:
            print("🤷 Didn't catch that.")
        except Exception as e:
            print(f"❗ Error: {e}")
